In [1]:
!file /content/drive/MyDrive/AImodels/spam_model.zip

/content/drive/MyDrive/AImodels/spam_model.zip: Zip archive data, at least v1.0 to extract, compression method=store


In [2]:
!unzip -o /content/drive/MyDrive/AImodels/spam_model.zip -d /content/

Archive:  /content/drive/MyDrive/AImodels/spam_model.zip
   creating: /content/spam_model/
  inflating: /content/spam_model/special_tokens_map.json  
  inflating: /content/spam_model/tokenizer_config.json  
  inflating: /content/spam_model/vocab.txt  
  inflating: /content/spam_model/config.json  
  inflating: /content/spam_model/tokenizer.json  
  inflating: /content/spam_model/model.safetensors  


In [3]:
!pip install transformers datasets evaluate torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [4]:
# 🔥 ONE CELL TO FURTHER TRAIN AND SAVE TO DRIVE

# 1. Install needed libraries (in case runtime is clean)
!pip install datasets transformers evaluate --quiet

# 2. Imports
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch

# 3. Load your existing tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("/content/spam_model")
model = AutoModelForSequenceClassification.from_pretrained("/content/spam_model")
model.train()  # Set to training mode

# 4. Load a small spam dataset
dataset = load_dataset("sms_spam")

# 5. Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example["sms"], truncation=True, padding="max_length")

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 6. Setup training arguments (save to Drive at the end!)
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/spam_model_refined",  # Save everything to Drive!
    per_device_train_batch_size=16,
    num_train_epochs=2,  # Train lightly
    weight_decay=0.01,
    logging_dir="/content/drive/MyDrive/logs",  # Save logs too if needed
    save_total_limit=1,
    save_strategy="epoch",
)

# 7. Setup Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

# 7.1 Disable wandb
!pip install wandb
import wandb
wandb.init(mode="disabled")
trainer.train()

# 8. Train the model
trainer.train()

# 9. Save model and tokenizer manually to Drive
model.save_pretrained("/content/drive/MyDrive/AImodels/spam_modelv2")
tokenizer.save_pretrained("/content/drive/MyDrive/AImodels/spam_modelv2")

print("✅ Model fine-tuned and safely saved to Drive!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/359k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

Map:   0%|          | 0/5574 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
500,0.020400


Step,Training Loss
500,0.010000


✅ Model fine-tuned and safely saved to Drive!
